In [ ]:
import warnings

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from backend.autoencoders import Autoencoder, train_predict_autoencoder

warnings.filterwarnings("ignore")

# Data Loading

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Load a DataFrame with a specific version of a CSV
df: pd.DataFrame = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "shiveshprakash/34-year-daily-stock-data/versions/1",
    "stock_data.csv",
)

# Drop useless columns or which we will create ourselves
df = df.drop(columns=["prev_day"])

# Display the first few rows of the dataframe
df.head()

# Data Cleaning and Preprocessing

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Convert "dt" to datetime format
df["dt"] = pd.to_datetime(df["dt"], format="%Y-%m-%d")

# Check data types
df.dtypes

# Data Analysis

### General Plots

In [ ]:
# Plot the S&P 500 over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=df["dt"], y=df["sp500"], label="S&P 500")
plt.title("S&P 500 Index Over Time")
plt.xlabel("Date")
plt.ylabel("S&P 500 Index")
plt.legend()
plt.show()

In [ ]:
# Visualize the relationship between S&P 500 and DJIA
sns.scatterplot(x="sp500", y="djia", data=df)
plt.title("S&P 500 vs DJIA")
plt.xlabel("S&P 500 Index")
plt.ylabel("DJIA Index")
plt.show()

### Correlation Analysis

In [ ]:
# Select only numeric columns for correlation analysis
numeric_df = df.select_dtypes(include=[np.number])

# Compute the correlation matrix
corr_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

# Create new features

### Aggregate Rolling Features

In [ ]:
windows = [7, 14, 30, 90, 365]  # Rolling window sizes (days)

for window in windows:
    df[f"sp500_mean_{window}"] = df["sp500"].rolling(window=window).mean()
    df[f"sp500_std_{window}"] = df["sp500"].rolling(window=window).std()
    df[f"sp500_volume_mean_{window}"] = df["sp500_volume"].rolling(window=window).mean()
    df[f"sp500_volume_std_{window}"] = df["sp500_volume"].rolling(window=window).std()

    df[f"djia_mean_{window}"] = df["djia"].rolling(window=window).mean()
    df[f"djia_std_{window}"] = df["djia"].rolling(window=window).std()
    df[f"djia_volume_mean_{window}"] = df["djia_volume"].rolling(window=window).mean()
    df[f"djia_volume_std_{window}"] = df["djia_volume"].rolling(window=window).std()

    df[f"hsi_mean_{window}"] = df["hsi"].rolling(window=window).mean()
    df[f"hsi_std_{window}"] = df["hsi"].rolling(window=window).std()

    df[f"vix_mean_{window}"] = df["vix"].rolling(window=window).mean()
    df[f"vix_std_{window}"] = df["vix"].rolling(window=window).std()

# Drop rows with NaN values introduced by rolling calculations
df = df.dropna()

# Display the updated DataFrame
df.head()

### Autoencoders

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create lagged features (temporary DataFrame)
lag_days = 365
lagged_df = pd.DataFrame()
for lag in range(1, lag_days + 1):
    lagged_df[f"sp500_lag_{lag}"] = df["sp500"].shift(lag)
    lagged_df[f"vix_lag_{lag}"] = df["vix"].shift(lag)
    lagged_df[f"sp500_volume_lag_{lag}"] = df["sp500_volume"].shift(lag)

# Drop rows with NaN values
lagged_df = lagged_df.dropna()
df = df.iloc[lag_days:].reset_index(drop=True)

# Normalize lagged features
scaler = MinMaxScaler()
X_lagged = scaler.fit_transform(lagged_df.values)
joblib.dump(scaler, "lagged_scaler.pkl")  # Save the scaler

# Clean up: Delete the temporary lagged features DataFrame
del lagged_df

# Get input dimensions
input_dim = X_lagged.shape[1]  # Number of lagged features

# Display the number of lagged features
print(f"Nr of lagged features: {input_dim}")

In [ ]:
# Initialize the Autoencoder
encoding_dim = 10  # Compressed representation size
autoencoder = Autoencoder(input_dim, encoding_dim)

# Train the autoencoder and get embeddings
trained_autoencoder, embeddings = train_predict_autoencoder(
    autoencoder,
    X_lagged,
    epochs=100,
    batch_size=256,
    lr=0.0005,
    l1_penalty=0.001,
    weight_decay=1e-5
)

In [ ]:
# Convert embeddings to DataFrame
embedding_df = pd.DataFrame(embeddings, columns=[f"embed_{i + 1}" for i in range(embeddings.shape[1])])

### Prepare the final df to train

##### Take the right data for training

In [ ]:
# Select only numeric columns for training
training_df = df.select_dtypes(include=[np.number])

# Convert to an ordered categorical column
if training_df["joblessness"].dtypes != "category":
    training_df["joblessness"] = pd.Categorical(
        training_df["joblessness"],
        categories=[1, 2, 3, 4],
        ordered=True
    )

##### Scale the df

In [ ]:
# Separate the "joblessness" column
joblessness = training_df["joblessness"]

# Select all columns except "joblessness"
columns_to_scale = training_df.drop(columns=["joblessness"]).columns

# Apply MinMaxScaler to the selected columns
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(training_df[columns_to_scale])
joblib.dump(scaler, "training_df_scaler.pkl")  # Save the scaler

# Create a DataFrame for the scaled data
training_df = pd.DataFrame(scaled_data, columns=columns_to_scale, index=training_df.index)

# Add back the "joblessness" column
training_df["joblessness"] = joblessness

##### Attach embeddings to the training DataFrame

In [ ]:
# Attach embeddings to the main DataFrame
training_df = pd.concat([training_df.reset_index(drop=True), embedding_df], axis=1)

# Display the final DataFrame with embeddings
training_df.head()

In [ ]:
print(list(training_df.columns))

# Train the models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd

days_to_predict = [1, 7, 14, 21, 28]

In [ ]:
# Step 1: Create targets for the selected future days
next_days_columns = []
for day in days_to_predict:
    next_day_col = f"sp500_next_{day}"
    training_df[next_day_col] = training_df["sp500"].shift(-day)
    next_days_columns.append(next_day_col)

# Drop rows where any target is NaN (due to shifting)
training_df = training_df.dropna()

# Step 2: Define features (X) and multiple targets (y)
X = training_df.drop(columns=next_days_columns)
y = training_df[next_days_columns]

# Step 3: Perform time-based train-test split
split_index = int(0.8 * len(X))  # 80% for training
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Step 4: Train the model (multi-output regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Step 5: Make predictions for the selected days
y_pred = model.predict(X_test)

# Step 6: Evaluate the model for each selected future day and store results
evaluation_results = {}  # Dictionary to store MSE and R² for each day

for i, day in enumerate(days_to_predict):
    # Align the predictions by shifting y_pred
    y_pred_aligned = pd.Series(y_pred[:, i], index=y_test.index).shift(-day).dropna()
    y_test_aligned = y_test.iloc[:-day, i]  # Drop last 'day' rows in y_test to align with predictions

    # Ensure both are the same length for evaluation
    assert len(y_pred_aligned) == len(y_test_aligned), f"Mismatch for Day {day}"

    # Calculate metrics
    mse = mean_squared_error(y_test_aligned, y_pred_aligned)
    r2 = r2_score(y_test_aligned, y_pred_aligned)

    # Store results
    evaluation_results[day] = {"MSE": mse, "R²": r2}

    # Print results
    print(f"Day {day}: Mean Squared Error: {mse:.6f}, R² Score: {r2:.6f}")

In [ ]:
# Plot predictions vs actual values for each selected day with metrics
for i, day in enumerate(days_to_predict):  # Loop through specified prediction horizons
    # Align the predictions by shifting y_pred
    y_pred_aligned = pd.Series(y_pred[:, i], index=y_test.index).shift(-day).dropna()

    # Extract stored metrics
    mse = evaluation_results[day]["MSE"]
    r2 = evaluation_results[day]["R²"]

    # Plot for the specific day
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.iloc[:, i].values, label=f"Actual Day {day}", alpha=0.8)
    plt.plot(y_pred_aligned.values, label=f"Predicted Day {day}", linestyle="--", alpha=0.8)

    # Add text box with MSE and R²
    textstr = f"MSE: {mse:.4f}\n    R²: {r2:.4f}"
    plt.text(0.05, 0.85, textstr, transform=plt.gca().transAxes,
             fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

    plt.title(f"Actual vs Predicted for Day {day}")
    plt.xlabel("Test Samples")
    plt.ylabel("S&P 500 Index")
    plt.legend()
    plt.grid()
    plt.show()

# Conclusion and Future Work
...